## Storing data on disk with SQLite

In [ ]:
!pip install exchange-calendars

In [1]:
import sqlite3

In [2]:
import exchange_calendars as xcals
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb

In [3]:
obb.user.preferences.output_type = "dataframe"

Function to fetch historical stock data for a given symbol and date range, and add a 'symbol' column

In [4]:
def get_stock_data(symbol, start_date=None, end_date=None):
    data = obb.equity.price.historical(
        symbol,
        start_date=start_date,
        end_date=end_date,
        provider="yfinance",
    )
    data.reset_index(inplace=True)
    data["symbol"] = symbol
    return data

Function to save the fetched stock data to an SQLite database

In [5]:
def save_data_range(symbol, conn, start_date=None, end_date=None):
    data = get_stock_data(symbol, start_date, end_date)
    data.to_sql("stock_data", conn, if_exists="append", index=False)

Function to save the stock data for the last trading session to an SQLite database

In [6]:
def save_last_trading_session(symbol, conn):
    today = pd.Timestamp.today()
    data = get_stock_data(symbol, today, today)
    data.to_sql("stock_data", conn, if_exists="append", index=False)

Establish a connection to the SQLite database

In [7]:
conn = sqlite3.connect("market_data.sqlite")

Save data for multiple stock symbols in the specified date range

In [8]:
for symbol in ["SPY", "QQQ", "DIA"]:
    save_data_range(symbol, conn=conn, start_date="2020-06-01", end_date="2023-01-01")

Read and display data for the stock symbol "SPY" from the database

In [9]:
df_1 = pd.read_sql_query("SELECT * from stock_data where symbol='SPY'", conn)
display(df_1)

,date,open,high,low,close,volume,split_ratio,dividend,capital_gains,symbol
0,2020-06-01,303.619995,306.209991,303.059998,305.549988,55758300,0.0,0.0,0.0,SPY
1,2020-06-02,306.549988,308.130005,305.100006,308.079987,74267200,0.0,0.0,0.0,SPY
2,2020-06-03,310.239990,313.220001,309.940002,312.179993,92567600,0.0,0.0,0.0,SPY
3,2020-06-04,311.109985,313.000000,309.079987,311.359985,75794400,0.0,0.0,0.0,SPY
4,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,0.0,0.0,SPY
...,...,...,...,...,...,...,...,...,...,...
1301,2022-12-23,379.649994,383.059998,378.029999,382.910004,59857300,0.0,0.0,0.0,SPY
1302,2022-12-27,382.790009,383.149994,379.649994,381.399994,51638200,0.0,0.0,0.0,SPY
1303,2022-12-28,381.329987,383.390015,376.420013,376.660004,70911500,0.0,0.0,0.0,SPY
1304,2022-12-29,379.630005,384.350006,379.079987,383.440002,66970900,0.0,0.0,0.0,SPY


Read and display data for "SPY" where the volume is greater than 100,000,000

In [10]:
df_2 = pd.read_sql_query(
    "SELECT * from stock_data where symbol='SPY' and volume > 100000000", conn
)
display(df_2)

,date,open,high,low,close,volume,split_ratio,dividend,capital_gains,symbol
0,2020-06-05,317.230011,321.269989,317.160004,319.339996,150524700,0.0,0.000,0.0,SPY
1,2020-06-11,311.459991,312.149994,300.010010,300.609985,209243600,0.0,0.000,0.0,SPY
2,2020-06-12,308.239990,309.079987,298.600006,304.209991,194678900,0.0,0.000,0.0,SPY
3,2020-06-15,298.019989,308.279999,296.739990,307.049988,135782700,0.0,0.000,0.0,SPY
4,2020-06-16,315.480011,315.640015,307.670013,312.959991,137627500,0.0,0.000,0.0,SPY
...,...,...,...,...,...,...,...,...,...,...
303,2022-12-13,410.220001,410.489990,399.070007,401.970001,123782500,0.0,0.000,0.0,SPY
304,2022-12-14,401.609985,405.500000,396.309998,399.399994,108111300,0.0,0.000,0.0,SPY
305,2022-12-15,394.299988,395.250000,387.890015,389.630005,117705900,0.0,0.000,0.0,SPY
306,2022-12-16,385.179993,386.579987,381.040009,383.269989,119858000,0.0,1.781,0.0,SPY


**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.